In [1]:
import pandas as pd
import xgboost as xgb
import lightgbm as lgb
import optuna
from collections import defaultdict

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso

from src.utils import get_kfold_data, convert_non_numeric_to_numeric, calculate_r2_score, calculate_metrics
from src.normalisation import Normaliser
from src.constants import *


c:\Users\wlyle\Desktop\DataScienceDataChallenge\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = pd.read_csv(DATA_PATH)

In [3]:
data.head()

,outcome,carat,cut,color,clarity,depth,table,price,x,y,...,a6,a7,a8,a9,a10,b6,b7,b8,b9,b10
0,-26.701232,1.14,Ideal,G,VS1,62.3,56.0,7948,6.73,6.70,...,0.168836,-0.273758,1.107832,1.247795,0.482344,0.489511,-0.321138,0.573382,0.446871,-1.990581
1,6.548093,0.38,Premium,H,VS2,60.5,59.0,898,4.69,4.66,...,-0.256549,0.315373,-0.030326,-0.114335,-1.059588,-1.761360,-1.343951,-1.002550,-0.225030,-0.446653
2,6.612562,0.50,Very Good,E,SI1,60.7,58.0,1351,5.09,5.13,...,-1.193327,-0.657307,-0.591726,-0.446856,-0.765286,-0.816544,-1.397794,-0.477130,0.810509,1.725131
3,-5.073562,0.70,Premium,D,SI1,61.2,58.0,2512,5.74,5.70,...,-1.740788,-1.778860,-0.825070,0.444932,1.173109,0.453606,-0.263440,0.246210,-0.850503,-0.412950
4,-14.436557,0.83,Ideal,G,SI2,62.4,54.0,2751,6.01,6.08,...,-0.859322,1.409268,0.861992,1.109063,-1.436722,-1.461618,0.081787,0.258087,0.851146,2.204813


Inspecting columns

In [4]:
# Find columns
all_columns = data.columns.tolist()
print(all_columns)

numeric_columns = data.select_dtypes(include=["number"]).columns.tolist()
numeric_columns.remove("outcome") # Remove the target column
print(numeric_columns)

non_numeric_columns = data.select_dtypes(exclude=["number"]).columns.tolist()
print(non_numeric_columns)

['outcome', 'carat', 'cut', 'color', 'clarity', 'depth', 'table', 'price', 'x', 'y', 'z', 'a1', 'a2', 'a3', 'a4', 'a5', 'b1', 'b2', 'b3', 'b4', 'b5', 'a6', 'a7', 'a8', 'a9', 'a10', 'b6', 'b7', 'b8', 'b9', 'b10']
['carat', 'depth', 'table', 'price', 'x', 'y', 'z', 'a1', 'a2', 'a3', 'a4', 'a5', 'b1', 'b2', 'b3', 'b4', 'b5', 'a6', 'a7', 'a8', 'a9', 'a10', 'b6', 'b7', 'b8', 'b9', 'b10']
['cut', 'color', 'clarity']


In [5]:
for non_numeric_column in non_numeric_columns:
    print(data[non_numeric_column].value_counts())

cut
Ideal        4040
Premium      2439
Very Good    2296
Good          925
Fair          300
Name: count, dtype: int64
color
G    2120
E    1873
F    1746
H    1506
D    1246
I     983
J     526
Name: count, dtype: int64
clarity
SI1     2408
VS2     2256
SI2     1743
VS1     1503
VVS2     951
VVS1     675
IF       318
I1       146
Name: count, dtype: int64


Converting non-numeric features to numerical features

In [6]:
data = convert_non_numeric_to_numeric(data=data)
print(data)

['G', 'E', 'F', 'H', 'D', 'I', 'J']
        outcome  carat  cut  clarity  depth  table  price     x     y     z  \
0    -26.701232   1.14    0        3   62.3   56.0   7948  6.73  6.70  4.18   
1      6.548093   0.38    1        4   60.5   59.0    898  4.69  4.66  2.83   
2      6.612562   0.50    2        5   60.7   58.0   1351  5.09  5.13  3.10   
3     -5.073562   0.70    1        5   61.2   58.0   2512  5.74  5.70  3.50   
4    -14.436557   0.83    0        6   62.4   54.0   2751  6.01  6.08  3.77   
...         ...    ...  ...      ...    ...    ...    ...   ...   ...   ...   
9995  10.718277   0.33    0        3   62.6   57.0   1002  4.42  4.40  2.76   
9996 -12.246698   1.01    4        5   69.5   55.0   4853  6.00  5.94  4.15   
9997  11.122516   0.52    2        6   57.9   61.0   1273  5.28  5.33  3.07   
9998 -24.730782   0.31    0        0   62.0   54.0    801  4.35  4.39  2.71   
9999   8.735755   0.37    2        5   59.9   59.0    649  4.68  4.70  2.81   

      ...      

Normalise data using each columns respective mean and std.

In [7]:
print(data)

        outcome  carat  cut  clarity  depth  table  price     x     y     z  \
0    -26.701232   1.14    0        3   62.3   56.0   7948  6.73  6.70  4.18   
1      6.548093   0.38    1        4   60.5   59.0    898  4.69  4.66  2.83   
2      6.612562   0.50    2        5   60.7   58.0   1351  5.09  5.13  3.10   
3     -5.073562   0.70    1        5   61.2   58.0   2512  5.74  5.70  3.50   
4    -14.436557   0.83    0        6   62.4   54.0   2751  6.01  6.08  3.77   
...         ...    ...  ...      ...    ...    ...    ...   ...   ...   ...   
9995  10.718277   0.33    0        3   62.6   57.0   1002  4.42  4.40  2.76   
9996 -12.246698   1.01    4        5   69.5   55.0   4853  6.00  5.94  4.15   
9997  11.122516   0.52    2        6   57.9   61.0   1273  5.28  5.33  3.07   
9998 -24.730782   0.31    0        0   62.0   54.0    801  4.35  4.39  2.71   
9999   8.735755   0.37    2        5   59.9   59.0    649  4.68  4.70  2.81   

      ...        b8        b9       b10  colour_G  

In [8]:
normaliser = Normaliser()
for column in numeric_columns:
    print(data[column])
    data[column] = normaliser.standardise(data[column])
    print("after", data[column])
    break

0       1.14
1       0.38
2       0.50
3       0.70
4       0.83
        ... 
9995    0.33
9996    1.01
9997    0.52
9998    0.31
9999    0.37
Name: carat, Length: 10000, dtype: float64
after 0       0.723643
1      -0.886369
2      -0.632156
3      -0.208469
4       0.066928
          ...   
9995   -0.992290
9996    0.448246
9997   -0.589788
9998   -1.034659
9999   -0.907553
Name: carat, Length: 10000, dtype: float64


In [9]:
print(data)

        outcome     carat  cut  clarity  depth  table  price     x     y  \
0    -26.701232  0.723643    0        3   62.3   56.0   7948  6.73  6.70   
1      6.548093 -0.886369    1        4   60.5   59.0    898  4.69  4.66   
2      6.612562 -0.632156    2        5   60.7   58.0   1351  5.09  5.13   
3     -5.073562 -0.208469    1        5   61.2   58.0   2512  5.74  5.70   
4    -14.436557  0.066928    0        6   62.4   54.0   2751  6.01  6.08   
...         ...       ...  ...      ...    ...    ...    ...   ...   ...   
9995  10.718277 -0.992290    0        3   62.6   57.0   1002  4.42  4.40   
9996 -12.246698  0.448246    4        5   69.5   55.0   4853  6.00  5.94   
9997  11.122516 -0.589788    2        6   57.9   61.0   1273  5.28  5.33   
9998 -24.730782 -1.034659    0        0   62.0   54.0    801  4.35  4.39   
9999   8.735755 -0.907553    2        5   59.9   59.0    649  4.68  4.70   

         z  ...        b8        b9       b10  colour_G  colour_E  colour_F  \
0     4.

In [10]:
kfold_data = get_kfold_data(data=data, k=NUM_FOLDS, reproducibility_seed=REPRODUCIBILITY_SEED)

Fold: 0/5
Train shape: (6400, 37) | 64.00%
Validation shape: (1600, 37) | 16.00%
Test shape: (2000, 37) | 20.00%

Fold: 1/5
Train shape: (6400, 37) | 64.00%
Validation shape: (1600, 37) | 16.00%
Test shape: (2000, 37) | 20.00%

Fold: 2/5
Train shape: (6400, 37) | 64.00%
Validation shape: (1600, 37) | 16.00%
Test shape: (2000, 37) | 20.00%

Fold: 3/5
Train shape: (6400, 37) | 64.00%
Validation shape: (1600, 37) | 16.00%
Test shape: (2000, 37) | 20.00%

Fold: 4/5
Train shape: (6400, 37) | 64.00%
Validation shape: (1600, 37) | 16.00%
Test shape: (2000, 37) | 20.00%



Define models and hyperparameter tuning objectives for each model

In [11]:
def objective(model_type, trial, x_train, y_train, x_val, y_val):
    if model_type == lgb.LGBMRegressor:
        parameters = {
                    "objective": "regression",
                    "metric": "rmse",
                    "n_estimators": 100,
                    "verbosity": -1,
                    "bagging_freq": 1,
                    "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
                    "num_leaves": trial.suggest_int("num_leaves", 2, 2**10),
                    "subsample": trial.suggest_float("subsample", 0.05, 1.0),
                    "colsample_bytree": trial.suggest_float("colsample_bytree", 0.05, 1.0),
                    "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 1, 100),
                    "seed": REPRODUCIBILITY_SEED
        }
    elif model_type == xgb.XGBRegressor:
        parameters = {
            "objective": "reg:squarederror",
            "eval_metric": "rmse",
            "n_estimators": 100,
            "eta": trial.suggest_float("eta", 1e-2, 0.2, log=True),
            "gamma": trial.suggest_float("gamma", 1e-8, 10, log=True),
            "max_depth": trial.suggest_int("max_depth", 3, 10),
            "min_child_weight": trial.suggest_int("min_child_weight", 1, 6),
            "subsample": trial.suggest_float("subsample", 0.5, 1.0),
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
            "seed": REPRODUCIBILITY_SEED
        }
    elif model_type == RandomForestRegressor:
        parameters = {
            "n_estimators": 100,
            "criterion": trial.suggest_categorical("criterion", ["absolute_error", "squared_error"]),
            "max_features": trial.suggest_categorical("max_features", ["sqrt", "log2"]),
            "max_depth": trial.suggest_int("max_depth", 3, 10),
            "min_samples_split": trial.suggest_int("min_samples_split", 2, 10),
            "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 10),
            "bootstrap": True,
            "oob_score": False,
            "n_jobs": -1,
            "random_state": REPRODUCIBILITY_SEED
        }
    elif model_type == GradientBoostingRegressor:
        parameters = {
            "n_estimators": 100,
            "loss": trial.suggest_categorical("loss", ["absolute_error", "squared_error", "huber", "quantile"]),
            "criterion": trial.suggest_categorical("criterion", ["friedman_mse", "squared_error"]),
            "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
            "subsample": trial.suggest_float("subsample", 0.05, 1.0),
            "max_depth": trial.suggest_int("max_depth", 3, 10),
            "min_samples_split": trial.suggest_int("min_samples_split", 2, 10),
            "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 10),
            "max_features": trial.suggest_categorical("max_features", ["sqrt", "log2"]),
            "max_leaf_nodes": trial.suggest_int("max_leaf_nodes", 2, 2**10),
            "random_state": REPRODUCIBILITY_SEED
        }
    elif model_type == AdaBoostRegressor:
        parameters = {
            "n_estimators": trial.suggest_int("n_estimators", 50, 100),
            "loss": trial.suggest_categorical("loss", ["linear", "square", "exponential"]),
            "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
            "random_state": REPRODUCIBILITY_SEED
        }
    elif model_type == Ridge:
        parameters = {
            "alpha": trial.suggest_float("alpha", 1e-3, 0.1, log=True),
            "solver": trial.suggest_categorical("solver", ["auto", "svd", "cholesky", "lsqr", "sparse_cg", "sag", "saga"]),
            "fit_intercept": trial.suggest_categorical("fit_intercept", [True, False]),
            "positive": False,
            "random_state": REPRODUCIBILITY_SEED
        }
    elif model_type == Lasso:
        parameters = {
            "alpha": trial.suggest_float("alpha", 1e-3, 0.1, log=True),
            "fit_intercept": trial.suggest_categorical("fit_intercept", [True, False]),
            "selection": trial.suggest_categorical("selection", ["cyclic", "random"]),
            "warm_start": trial.suggest_categorical("warm_start", [True, False]),
            "random_state": REPRODUCIBILITY_SEED
        }
    elif model_type == LinearRegression:
        parameters = {
            "fit_intercept": trial.suggest_categorical("fit_intercept", [True, False]),
        }
    
    model = model_type(**parameters) # Create the model
    model.fit(x_train, y_train)
    predictions = model.predict(x_val)
    metrics = calculate_metrics(targets=y_val, preds=predictions)
    rmse = metrics["rmse"]
    return rmse


models = {
        # "linear_regression": LinearRegression,
        # "lasso": Lasso,
        "ridge": Ridge,
        # "xgb": xgb.XGBRegressor,
        # "random_forest": RandomForestRegressor,
        # "gradient_boosting": GradientBoostingRegressor,
        # "ada_boost": AdaBoostRegressor,
        # "lgbm": lgb.LGBMRegressor
        }

In [12]:
# Train + Validate models
metrics = ["mae", "mse", "rmse", "pcc", "spearman_r", "r2_score"]
model_scores = {model_name: defaultdict(list) for model_name in models.keys()}

for fold in range(NUM_FOLDS):
    fold_data = kfold_data[fold]
     
    # Extract data
    train_data = fold_data["train"]
    val_data = fold_data["val"]
    test_data = kfold_data[fold]["test"]

    train_y = train_data["outcome"]
    val_y = val_data["outcome"]
    test_y = test_data["outcome"]

    train_x = train_data.drop(columns=["outcome"])
    val_x = val_data.drop(columns=["outcome"])
    test_x = test_data.drop(columns=["outcome"])

    # print(f"Fold {fold+1}/{NUM_FOLDS}")
    # print(f"Train data shape: {train_x.shape} | Train target shape: {train_y.shape}")
    # print(f"Val data shape: {val_x.shape} | Val target shape: {val_y.shape}")
    # print(f"Test data shape: {test_x.shape} | Test target shape: {test_y.shape}")

    # Train model
    for model_name, model in models.items():
        study = optuna.create_study(direction="minimize")
        study.optimize(lambda trial: objective(trial=trial, 
                                               model_type=model, 
                                               x_train=train_x, 
                                               y_train=train_y, 
                                               x_val=val_x, 
                                               y_val=val_y
                                               ), n_trials=30)
        
        # Train model with best hyperparameters
        best_fold_params = study.best_params
        model = model(**best_fold_params)
        model.fit(train_x, train_y)
        preds = model.predict(val_x)

        metrics = calculate_metrics(targets=val_y, preds=preds)
        mae = metrics["mae"]
        mse = metrics["mse"]
        rmse = metrics["rmse"]
        pcc = metrics["pcc"]
        spearman_r = metrics["spearman_r"]
        r2_score = metrics["r2_score"]

        for metric in metrics:
            model_scores[model_name][metric].append(metrics[metric])

        print(f"Fold: {fold+1}/{NUM_FOLDS}")
        print(f"Model name: {model_name}")
        print(f"MAE: {mae}")
        print(f"MSE: {mse}")
        print(f"RMSE: {rmse}")
        print(f"PCC: {pcc}")
        print(f"Spearman R: {spearman_r}")
        print(f"R2 Score: {r2_score}")
        print()

# Compute average scores
for model_name, model_metrics in model_scores.items():
    for metric, scores in model_metrics.items():
        model_scores[model_name][metric] = sum(scores) / len(scores)
    model_scores[model_name] = dict(model_scores[model_name])
print(model_scores)

[I 2025-02-16 15:44:05,279] A new study created in memory with name: no-name-71b3882d-f9fa-42b3-b884-342aa40bcf20
c:\Users\wlyle\Desktop\DataScienceDataChallenge\.venv\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
[I 2025-02-16 15:44:06,277] Trial 0 finished with value: 12.948190315309555 and parameters: {'alpha': 0.004648079654055129, 'solver': 'saga', 'fit_intercept': True}. Best is trial 0 with value: 12.948190315309555.
[I 2025-02-16 15:44:06,287] Trial 1 finished with value: 10.946969721581594 and parameters: {'alpha': 0.04680742735113145, 'solver': 'svd', 'fit_intercept': False}. Best is trial 1 with value: 10.946969721581594.
[I 2025-02-16 15:44:06,293] Trial 2 finished with value: 10.916909495021754 and parameters: {'alpha': 0.004106965439608222, 'solver': 'sparse_cg', 'fit_intercept': True}. Best is trial 2 with value: 10.916909495021754.
[I 2025-02-16 15:44:06,298] Trial

Fold: 1/5
Model name: ridge
MAE: 8.776012714253206
MSE: 119.12145871321823
RMSE: 10.9142777458345
PCC: 0.5462868819947289
Spearman R: 0.5695827010088677
R2 Score: 0.297964776661807



c:\Users\wlyle\Desktop\DataScienceDataChallenge\.venv\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
[I 2025-02-16 15:44:11,298] Trial 2 finished with value: 12.565087906251122 and parameters: {'alpha': 0.05517392030235027, 'solver': 'saga', 'fit_intercept': False}. Best is trial 1 with value: 10.758844617004273.
[I 2025-02-16 15:44:11,306] Trial 3 finished with value: 10.829292088556562 and parameters: {'alpha': 0.06252286819365524, 'solver': 'svd', 'fit_intercept': False}. Best is trial 1 with value: 10.758844617004273.
[I 2025-02-16 15:44:11,310] Trial 4 finished with value: 11.799601032710045 and parameters: {'alpha': 0.023385809396966405, 'solver': 'lsqr', 'fit_intercept': True}. Best is trial 1 with value: 10.758844617004273.
c:\Users\wlyle\Desktop\DataScienceDataChallenge\.venv\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached w

Fold: 2/5
Model name: ridge
MAE: 8.572073281281346
MSE: 115.66458947111536
RMSE: 10.754747299268141
PCC: 0.5251901941505763
Spearman R: 0.5521689012378521
R2 Score: 0.2750354343071011



c:\Users\wlyle\Desktop\DataScienceDataChallenge\.venv\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
[I 2025-02-16 15:44:16,123] Trial 3 finished with value: 12.67636366835567 and parameters: {'alpha': 0.030780208622879224, 'solver': 'sag', 'fit_intercept': False}. Best is trial 2 with value: 10.768617055789345.
[I 2025-02-16 15:44:16,132] Trial 4 finished with value: 10.768616746210544 and parameters: {'alpha': 0.01885136948915723, 'solver': 'svd', 'fit_intercept': True}. Best is trial 4 with value: 10.768616746210544.
[I 2025-02-16 15:44:16,136] Trial 5 finished with value: 12.092737816706338 and parameters: {'alpha': 0.02351309830980028, 'solver': 'lsqr', 'fit_intercept': False}. Best is trial 4 with value: 10.768616746210544.
[I 2025-02-16 15:44:16,142] Trial 6 finished with value: 10.767711187572155 and parameters: {'alpha': 0.05333957135075572, 'solver': 'sparse_cg', 'fit_int

Fold: 3/5
Model name: ridge
MAE: 8.640715656164911
MSE: 115.91251126635788
RMSE: 10.766267285663954
PCC: 0.538994754679601
Spearman R: 0.5713171616473289
R2 Score: 0.28930948238516596



c:\Users\wlyle\Desktop\DataScienceDataChallenge\.venv\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
[I 2025-02-16 15:44:21,445] Trial 4 finished with value: 12.6156397083005 and parameters: {'alpha': 0.0815033955737289, 'solver': 'saga', 'fit_intercept': True}. Best is trial 1 with value: 10.480641428643567.
[I 2025-02-16 15:44:21,448] Trial 5 finished with value: 11.598384557107469 and parameters: {'alpha': 0.0029413944393056357, 'solver': 'lsqr', 'fit_intercept': True}. Best is trial 1 with value: 10.480641428643567.
[I 2025-02-16 15:44:21,454] Trial 6 finished with value: 10.481317946474032 and parameters: {'alpha': 0.026462724399111178, 'solver': 'sparse_cg', 'fit_intercept': True}. Best is trial 1 with value: 10.480641428643567.
[I 2025-02-16 15:44:21,464] Trial 7 finished with value: 10.561319242693655 and parameters: {'alpha': 0.04699448392042688, 'solver': 'svd', 'fit_inte

Fold: 4/5
Model name: ridge
MAE: 8.275275207338948
MSE: 109.84297306583547
RMSE: 10.480599842844658
PCC: 0.5648406527524452
Spearman R: 0.58996920311297
R2 Score: 0.31694618854879775



c:\Users\wlyle\Desktop\DataScienceDataChallenge\.venv\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
[I 2025-02-16 15:44:25,373] Trial 0 finished with value: 12.723142421262807 and parameters: {'alpha': 0.013603112503737629, 'solver': 'saga', 'fit_intercept': False}. Best is trial 0 with value: 12.723142421262807.
c:\Users\wlyle\Desktop\DataScienceDataChallenge\.venv\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
[I 2025-02-16 15:44:26,358] Trial 1 finished with value: 12.70161223716092 and parameters: {'alpha': 0.01437977428783033, 'solver': 'saga', 'fit_intercept': True}. Best is trial 1 with value: 12.70161223716092.
c:\Users\wlyle\Desktop\DataScienceDataChallenge\.venv\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which mea

Fold: 5/5
Model name: ridge
MAE: 8.58120062670553
MSE: 116.64400181868884
RMSE: 10.800185267794662
PCC: 0.5355513509482475
Spearman R: 0.556710000667969
R2 Score: 0.2861560820450474

{'ridge': {'mae': 8.569055497148788, 'mse': 115.43710686704317, 'rmse': 10.743215488281184, 'pcc': 0.5421727669051197, 'spearman_r': 0.5679495935349975, 'r2_score': 0.29308239278958387}}
